In [2]:
import numpy as np
import pandas as pd



In [4]:
! (Get-Content ml-10M100K\ratings.dat) -replace '::', "`t" | Set-Content ml-10M100K\ratings.dat


No se esperaba -replace en este momento.


In [3]:
# Convert MovieLens data to binary using numpy_to_binary function
def movie_lens_to_binary(input_file, output_file):
    # Load MovieLens data using Pandas
    ratings = pd.read_csv(input_file, sep='\t', header=None,
                          names=['userId', 'movieId', 'rating', 'rating_timestamp'])
    # Convert to NumPy array
    np_data = np.array(ratings[['userId', 'movieId', 'rating']])
    # Write to binary file
    with open(output_file, "wb") as bin_file:
        bin_file.write(np_data.astype(np.int32).tobytes())
movie_lens_to_binary('ml-10M100K/ratings.dat', 'ml-10M100K/output_binary.bin')

ValueError: invalid literal for int() with base 10: '1::122::5::838985046'

In [3]:
import numpy as np
import pandas as pd

def binary_to_pandas(bin_file, num_rows=10):
    # Read binary data into NumPy array
    with open(bin_file, 'rb') as f:
        binary_data = f.read()

    # Convert binary data back to NumPy array
    np_data = np.frombuffer(binary_data, dtype=np.int32).reshape(-1, 3)  # Assuming 3 columns

    # Convert NumPy array to Pandas DataFrame
    df = pd.DataFrame(np_data, columns=['userId', 'movieId', 'rating'])

    # Display the equivalent of ratings.head(10)
    print(df.head(num_rows))

# Usage
binary_to_pandas('ml-10M100K/output_binary.bin', num_rows=10)


   userId  movieId  rating
0       1      122       5
1       1      185       5
2       1      231       5
3       1      292       5
4       1      316       5
5       1      329       5
6       1      355       5
7       1      356       5
8       1      362       5
9       1      364       5


In [4]:
import numpy as np
import pandas as pd

def binary_to_pandas_with_stats(bin_file, num_rows=10):
    # Read binary data into NumPy array
    with open(bin_file, 'rb') as f:
        binary_data = f.read()
    # Convert binary data back to NumPy array
    np_data = np.frombuffer(binary_data, dtype=np.int32).reshape(-1, 3)  # Assuming 3 columns
    # Convert NumPy array to Pandas DataFrame
    df = pd.DataFrame(np_data, columns=['userId', 'movieId', 'rating'])
    # Calculate max and min values for 'userId'
    userId_max = df['userId'].max()
    userId_min = df['userId'].min()
    num_rows_df = len(df.index)
    return userId_max, userId_min, num_rows_df
# Usage
userId_max, userId_min, num_rows_df = binary_to_pandas_with_stats('ml-10M100K/output_binary.bin', num_rows=10)

print(f"Maximum userId: {userId_max}")
print(f"Minimum userId: {userId_min}")
print(f"Number of rows: {num_rows_df}")

Maximum userId: 71567
Minimum userId: 1
Number of rows: 10000054


In [5]:
#it takes 6 seconds
import numpy as np
import pandas as pd
def binary_to_pandas_with_stats(bin_file, num_rows=10):
    # Read binary data into NumPy array
    with open(bin_file, 'rb') as f:
        binary_data = f.read()
    # Convert binary data back to NumPy array
    np_data = np.frombuffer(binary_data, dtype=np.int32).reshape(-1, 3)  # Assuming 3 columns
    # Convert NumPy array to Pandas DataFrame
    df = pd.DataFrame(np_data, columns=['userId', 'movieId', 'rating'])
    return df
def consolidate_data(df):
    # Group by 'userId' and aggregate 'movieId' and 'rating' into lists
    consolidated = df.groupby('userId', as_index=False).agg({
        'movieId': lambda x: x.tolist(),
        'rating': lambda x: x.tolist()
    })
    return consolidated
df = binary_to_pandas_with_stats('ml-10M100K/output_binary.bin', num_rows=10)
# Consolidate data
consolidated = consolidate_data(df)
print("Consolidated data:")
print(consolidated)


Consolidated data:
       userId                                            movieId  \
0           1  [122, 185, 231, 292, 316, 329, 355, 356, 362, ...   
1           2  [110, 151, 260, 376, 539, 590, 648, 719, 733, ...   
2           3  [110, 151, 213, 590, 1148, 1246, 1252, 1276, 1...   
3           4  [21, 34, 39, 110, 150, 153, 161, 165, 208, 231...   
4           5  [1, 7, 25, 28, 30, 32, 47, 52, 57, 58, 85, 111...   
...       ...                                                ...   
69873   71563  [145, 196, 293, 356, 527, 733, 780, 1088, 1092...   
69874   71564  [3, 19, 62, 104, 217, 231, 235, 260, 288, 315,...   
69875   71565  [6, 21, 34, 73, 110, 111, 246, 260, 265, 318, ...   
69876   71566  [1, 17, 22, 25, 32, 34, 36, 50, 58, 62, 110, 1...   
69877   71567  [32, 110, 196, 256, 260, 316, 442, 480, 589, 7...   

                                                  rating  
0      [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...  
1      [5, 3, 5, 3, 3, 5, 2, 3, 3, 3, 3, 3, 2,

In [6]:
#16 seg
import numpy as np
import pandas as pd

def binary_to_pandas_with_stats(bin_file, num_rows=10):
    # Read binary data into NumPy array
    with open(bin_file, 'rb') as f:
        binary_data = f.read()
    # Convert binary data back to NumPy array
    np_data = np.frombuffer(binary_data, dtype=np.int32).reshape(-1, 3)  # Assuming 3 columns
    # Convert NumPy array to Pandas DataFrame
    df = pd.DataFrame(np_data, columns=['userId', 'movieId', 'rating'])
    return df
def consolidate_data(df):
    # Group by 'userId' and 'movieId' and calculate the mean of 'rating'
    consolidated_df = df.groupby(['userId', 'movieId'])['rating'].mean().unstack()
    return consolidated_df
df = binary_to_pandas_with_stats('ml-10M100K/output_binary.bin', num_rows=10)

# Consolidate data
consolidated_df = consolidate_data(df)
print("Consolidated data:")
print(consolidated_df)


Consolidated data:
movieId  1      2      3      4      5      6      7      8      9      10     \
userId                                                                          
1          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5          1.0    NaN    NaN    NaN    NaN    NaN    3.0    NaN    NaN    NaN   
...        ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
71563      NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
71564      NaN    NaN    2.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
71565      NaN    NaN    NaN    NaN    NaN    3.0    NaN    NaN    NaN    NaN   
71566      5.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
71567    

In [12]:
#it takes 32 seconds
#comparate
import pandas as pd
import numpy as np
from scipy.spatial.distance import cityblock

def computeNearestNeighbor(dataframe, target_user, distance_metric=cityblock):
    distances = np.zeros(len(dataframe))  # Inicializa un array NumPy
    # Itera sobre cada fila (usuario) en el DataFrame
    for i, (index, row) in enumerate(dataframe.iterrows()):
        if index == target_user:
            continue  # Omite el propio usuario objetivo
        # Calcula la distancia entre el usuario objetivo y el usuario actual
        distance = distance_metric(dataframe.loc[target_user].fillna(0), row.fillna(0))
        distances[i] = distance
    # Obtiene los índices que ordenarían el array y luego ordena las distancias y los índices de los usuarios en consecuencia
    sorted_indices = np.argsort(distances)
    sorted_distances = distances[sorted_indices]
    return list(zip(dataframe.index[sorted_indices], sorted_distances))
# Ejemplo de uso
# Suponiendo que tu DataFrame se llama 'consolidated_df'
target_user_id = 1
neighbors = computeNearestNeighbor(consolidated_df, target_user_id)
# Imprime los vecinos más cercanos y sus distancias
print("Nearest Neighbors for User {}: {}".format(target_user_id, neighbors))

Nearest Neighbors for User 1: [(1, 0.0), (26370, 94.0), (62696, 107.0), (49862, 110.0), (48146, 110.0), (13496, 110.0), (62815, 110.0), (63381, 111.0), (59001, 112.0), (54370, 112.0), (5729, 114.0), (38426, 114.0), (59245, 115.0), (56471, 115.0), (69038, 115.0), (55121, 115.0), (58401, 115.0), (57462, 117.0), (39211, 117.0), (6322, 117.0), (55655, 117.0), (33034, 118.0), (62965, 118.0), (23747, 118.0), (67653, 118.0), (32295, 118.0), (23783, 119.0), (43987, 119.0), (48176, 119.0), (65934, 119.0), (65547, 120.0), (54739, 120.0), (11211, 120.0), (8920, 120.0), (67806, 121.0), (43786, 121.0), (52891, 121.0), (10053, 121.0), (2694, 121.0), (68430, 122.0), (44259, 122.0), (50852, 122.0), (26225, 122.0), (59360, 122.0), (69474, 122.0), (15515, 122.0), (65875, 123.0), (4444, 123.0), (33934, 123.0), (11982, 123.0), (22871, 123.0), (54985, 123.0), (60112, 123.0), (54649, 123.0), (33261, 123.0), (71452, 124.0), (61976, 124.0), (4042, 124.0), (56697, 124.0), (2868, 124.0), (25925, 124.0), (37966,

In [11]:
import torch
import pandas as pd
from scipy.spatial.distance import cityblock

def computeNearestNeighbor(dataframe, target_user,):
    # Convert DataFrame to PyTorch tensors
    target_user_tensor = torch.tensor(dataframe.loc[target_user].fillna(0).values, dtype=torch.float32)
    user_data_tensor = torch.tensor(dataframe.fillna(0).values, dtype=torch.float32)

    # Calculate distances using PyTorch
    distances = torch.norm(user_data_tensor - target_user_tensor, p=1, dim=1).numpy()

    # Get the indices that would sort the array, and then sort the distances accordingly
    sorted_indices = distances.argsort()
    sorted_distances = distances[sorted_indices]

    return list(zip(dataframe.index[sorted_indices], sorted_distances))

# Example usage
# Assuming your DataFrame is named 'consolidated_df'
target_user_id = 1
neighbors = computeNearestNeighbor(consolidated_df, target_user_id)
# Print the nearest neighbors and their distances
print("Nearest Neighbors for User {}: {}".format(target_user_id, neighbors))


Nearest Neighbors for User 1: [(1, 0.0), (26370, 94.0), (62696, 107.0), (49862, 110.0), (48146, 110.0), (13496, 110.0), (62815, 110.0), (63381, 111.0), (59001, 112.0), (54370, 112.0), (5729, 114.0), (38426, 114.0), (59245, 115.0), (56471, 115.0), (69038, 115.0), (55121, 115.0), (58401, 115.0), (57462, 117.0), (39211, 117.0), (6322, 117.0), (55655, 117.0), (33034, 118.0), (62965, 118.0), (23747, 118.0), (67653, 118.0), (32295, 118.0), (23783, 119.0), (43987, 119.0), (48176, 119.0), (65934, 119.0), (65547, 120.0), (54739, 120.0), (11211, 120.0), (8920, 120.0), (67806, 121.0), (43786, 121.0), (52891, 121.0), (10053, 121.0), (2694, 121.0), (68430, 122.0), (44259, 122.0), (50852, 122.0), (26225, 122.0), (59360, 122.0), (69474, 122.0), (15515, 122.0), (65875, 123.0), (4444, 123.0), (33934, 123.0), (11982, 123.0), (22871, 123.0), (54985, 123.0), (60112, 123.0), (54649, 123.0), (33261, 123.0), (71452, 124.0), (61976, 124.0), (4042, 124.0), (56697, 124.0), (2868, 124.0), (25925, 124.0), (37966,

In [10]:
import torch

def computeNearestNeighbor(dataframe, target_user):
    # Convert DataFrame to PyTorch tensors
    target_user_tensor = torch.tensor(dataframe.loc[target_user].fillna(0).values, dtype=torch.float32)
    user_data_tensor = torch.tensor(dataframe.fillna(0).values, dtype=torch.float32)

    # Move tensors to GPU if available
    if torch.cuda.is_available():
        target_user_tensor = target_user_tensor.cuda()
        user_data_tensor = user_data_tensor.cuda()

    # Calculate distances using PyTorch
    distances = torch.norm(user_data_tensor - target_user_tensor, p=1, dim=1).cpu().numpy()

    # Get the indices that would sort the array, and then sort the distances accordingly
    sorted_indices = distances.argsort()
    sorted_distances = distances[sorted_indices]

    return list(zip(dataframe.index[sorted_indices], sorted_distances))

# Example usage
# Assuming your DataFrame is named 'consolidated_df'
target_user_id = 1
neighbors = computeNearestNeighbor(consolidated_df, target_user_id)
# Print the nearest neighbors and their distances
print("Nearest Neighbors for User {}: {}".format(target_user_id, neighbors))


Nearest Neighbors for User 1: [(1, 0.0), (26370, 94.0), (62696, 107.0), (49862, 110.0), (48146, 110.0), (13496, 110.0), (62815, 110.0), (63381, 111.0), (59001, 112.0), (54370, 112.0), (5729, 114.0), (38426, 114.0), (59245, 115.0), (56471, 115.0), (69038, 115.0), (55121, 115.0), (58401, 115.0), (57462, 117.0), (39211, 117.0), (6322, 117.0), (55655, 117.0), (33034, 118.0), (62965, 118.0), (23747, 118.0), (67653, 118.0), (32295, 118.0), (23783, 119.0), (43987, 119.0), (48176, 119.0), (65934, 119.0), (65547, 120.0), (54739, 120.0), (11211, 120.0), (8920, 120.0), (67806, 121.0), (43786, 121.0), (52891, 121.0), (10053, 121.0), (2694, 121.0), (68430, 122.0), (44259, 122.0), (50852, 122.0), (26225, 122.0), (59360, 122.0), (69474, 122.0), (15515, 122.0), (65875, 123.0), (4444, 123.0), (33934, 123.0), (11982, 123.0), (22871, 123.0), (54985, 123.0), (60112, 123.0), (54649, 123.0), (33261, 123.0), (71452, 124.0), (61976, 124.0), (4042, 124.0), (56697, 124.0), (2868, 124.0), (25925, 124.0), (37966,